In [ ]:
import librosa
import numpy as np
from PIL import Image
import os
import glob
import time

def generate_amplitude_image(file_path, grid_size=64):
    """
    Generates a grayscale image from an audio file based on the
    average amplitude of its segments.
    
    Args:
        file_path (str): The full path to the audio file.
        grid_size (int): The dimension of the grid (e.g., 16x16).
        
    Returns:
        PIL.Image: A 3-channel (RGB) grayscale image, or None if error.
    """
    
    num_segments = grid_size * grid_size # 16*16 = 256
    
    try:
        # 1. Get the audio file
        y, sr = librosa.load(file_path, sr=None)
    except Exception as e:
        print(f"  [Error] Skipping {os.path.basename(file_path)}: {e}")
        return None
    
    # Check for silent/empty files
    if len(y) == 0:
        print(f"  [Warning] Skipping empty file: {os.path.basename(file_path)}")
        return None
        
    # We use the absolute value for amplitude
    y_abs = np.abs(y)
    
    # 2. Divide it into 256 equal segments
    segments = np.array_split(y_abs, num_segments)
    
    # 3. Calculate the average amplitude for each segment
    avg_amplitudes = [np.mean(segment) for segment in segments]
    amplitudes_array = np.array(avg_amplitudes)
    
    # 4. Normalize the amplitudes to a 0-255 scale
    min_amp = np.min(amplitudes_array)
    max_amp = np.max(amplitudes_array)
    range_amp = max_amp - min_amp
    
    if range_amp == 0:
        # Avoid division by zero if the audio is silent or constant
        normalized_amplitudes = np.zeros(num_segments, dtype=np.uint8)
    else:
        # Min-max normalization
        normalized_amplitudes = ((amplitudes_array - min_amp) / range_amp) * 255.0
        normalized_amplitudes = normalized_amplitudes.astype(np.uint8)
    
    # 5. & 6. Arrange pixels in a row-wise grid
    
    # Reshape the 1D array of 256 values into a 16x16 grid
    pixel_grid_1channel = normalized_amplitudes.reshape(grid_size, grid_size)
    
    # Create a 3-channel RGB image where R=G=B=y
    pixel_grid_3channel = np.stack([pixel_grid_1channel] * 3, axis=-1)
    
    # Create the image from the 3-channel NumPy array
    img = Image.fromarray(pixel_grid_3channel, 'RGB')
    
    return img

def process_all_genres(base_audio_dir, base_output_dir, grid_size=64):
    """
    Finds all genre sub-folders, loops through all .wav files,
    generates images, and saves them in a mirrored directory structure.
    """
    
    print(f"Starting batch processing...")
    print(f"Input directory:  {base_audio_dir}")
    print(f"Output directory: {base_output_dir}")
    print("---")
    
    start_time = time.time()
    total_files_processed = 0
    
    # Find all subdirectories (genres) in the base audio directory
    try:
        genre_dirs = [d.name for d in os.scandir(base_audio_dir) if d.is_dir()]
    except FileNotFoundError:
        print(f"ERROR: Input directory not found: {base_audio_dir}")
        print("Please check your 'base_audio_dir' variable.")
        return

    if not genre_dirs:
        print(f"ERROR: No genre sub-folders found in {base_audio_dir}.")
        return

    print(f"Found {len(genre_dirs)} genres: {genre_dirs}")

    # Loop over each genre folder
    for genre in genre_dirs:
        input_genre_path = os.path.join(base_audio_dir, genre)
        output_genre_path = os.path.join(base_output_dir, genre)
        
        # 1. Create the output directory (e.g., "amplitude_images/classical")
        os.makedirs(output_genre_path, exist_ok=True)
        
        # 2. Find all .wav files in the input genre directory
        audio_files = glob.glob(os.path.join(input_genre_path, "*.wav"))
        
        if not audio_files:
            print(f"\nNo .wav files found for genre: {genre}. Skipping.")
            continue

        print(f"\nProcessing {len(audio_files)} files for genre: {genre}...")
        files_in_genre = 0
        
        # 3. Loop over each file, generate image, and save
        for file_path in audio_files:
            # Generate the image
            amplitude_image = generate_amplitude_image(file_path, grid_size)
            
            if amplitude_image:
                # Create a new filename
                # e.g., "classical.00001.wav" -> "classical.00001.png"
                base_filename = os.path.basename(file_path)
                filename_no_ext = os.path.splitext(base_filename)[0]
                output_filename = f"{filename_no_ext}.png"
                
                # Create the full output path
                output_path = os.path.join(output_genre_path, output_filename)
                
                # Save the image
                amplitude_image.save(output_path)
                files_in_genre += 1

        print(f"Finished {genre}: {files_in_genre} images saved.")
        total_files_processed += files_in_genre

    end_time = time.time()
    print("---")
    print(f"✅ All processing complete.")
    print(f"Total files processed: {total_files_processed}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    print(f"Your images are saved in the '{base_output_dir}' directory.")


# --- 🚀 HOW TO RUN ---
if __name__ == "__main__":
    
    # 1. Define your main audio directory
    # This is the folder that CONTAINS "classical", "blues", "rock", etc.
    # ****** PLEASE UPDATE THIS PATH ******
    base_audio_dir = "Data/genres_original/" 
    
    # 2. Define where you want to save the new images
    # A new folder "amplitude_images" will be created.
    base_output_dir = "amplitude_images_64/"
    
    # 3. Run the processing
    process_all_genres(base_audio_dir, base_output_dir, grid_size=64)

In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- 1. Configuration ---
# This MUST match the 'base_output_dir' from your generation script
IMAGE_DIR = "amplitude_images_64/" 
CATEGORIES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'] # Make sure this matches your folder names
IMG_SIZE = 64 # This MUST match the 'grid_size' from your script

# --- 2. Load Data ---
print(f"Loading {IMG_SIZE}x{IMG_SIZE} images from '{IMAGE_DIR}'...")
data = []
labels = []

for i, category in enumerate(CATEGORIES):
    path = os.path.join(IMAGE_DIR, category)
    try:
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            
            # Load the image in COLOR (since your script saved as RGB)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
            
            # Resize just in case, though your script should handle this
            img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append(img_resized)
            labels.append(i) # 0 for blues, 1 for rock, 2 for nature
    except Exception as e:
        print(f"Error loading images from {path}: {e}")
        print("Please double-check your 'IMAGE_DIR' and 'CATEGORIES'")

if not data:
    print("\nError: No data loaded. Stopping.")
else:
    # --- 3. Preprocessing ---
    # Convert to numpy arrays
    X = np.array(data)
    y = np.array(labels)

    # Normalize pixel values from 0-255 to 0.0-1.0
    # This is critical for neural network performance
    X = X / 255.0

    # --- 4. Split Data (80% train, 20% test) ---
    # stratify=y ensures the 80/20 split has a fair mix of all 3 classes
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"\nTotal samples loaded: {len(X)}")
    print(f"Training data shape: {X_train.shape}") # Should be (240, 64, 64, 3)
    print(f"Testing data shape: {X_test.shape}")   # Should be (60, 64, 64, 3)

    # --- 5. Build the CNN Model ---
    # For 64x64 images, we'll use two Conv/Pool blocks
    
    model = tf.keras.models.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))) 
    
    # Block 1:
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    # Block 2:
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    # Flatten: Squashes 2D data to 1D
    model.add(tf.keras.layers.Flatten())
    
    # Hidden layer:
    model.add(tf.keras.layers.Dense(128, activation='relu')) # Increased size for 64x64
    
    # Output layer: 3 neurons (1 per class), softmax for probabilities
    model.add(tf.keras.layers.Dense(len(CATEGORIES), activation='softmax'))

    model.summary()

    # --- 6. Compile the Model ---
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy', # Use this for integer labels (0, 1, 2)
        metrics=['accuracy']
    )

    # --- 7. Train the Model ---
    print("\nTraining model...")
    history = model.fit(
        X_train, y_train,
        epochs=30, # 30-50 epochs is a good starting point
        validation_data=(X_test, y_test),
        verbose=1
    )

    # --- 8. Evaluate ---
    print("\nEvaluating on test data...")
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print("---" * 10)
    print(f"✅ Final Test Accuracy: {test_acc * 100:.2f}%")
    print("---" * 10)

Loading 64x64 images from 'amplitude_images_64/'...

Total samples loaded: 999
Training data shape: (799, 64, 64, 3)
Testing data shape: (200, 64, 64, 3)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,442 (6.20 MB)

 Trainable params: 1,626,442 (6.20 MB)

 Non-trainable params: 0 (0.00 B)


Training model...
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.0876 - loss: 2.3160 - val_accuracy: 0.1550 - val_loss: 2.2720
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.1577 - loss: 2.2404 - val_accuracy: 0.1600 - val_loss: 2.2349
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.2641 - loss: 2.0821 - val_accuracy: 0.2250 - val_loss: 2.1076
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.3655 - loss: 1.8262 - val_accuracy: 0.2100 - val_loss: 2.1837
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.4756 - loss: 1.5453 - val_accuracy: 0.2300 - val_loss: 2.1319
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6483 - loss: 1.1992 - val_accuracy: 0.2550 - val_loss: 2.2376
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.7559 - loss: 0.8286 - val_accuracy: 0.2950 - val_loss: 2.4370
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.8598 - loss: 0.5701 - val_a

In [3]:
import librosa
import numpy as np
from PIL import Image
import os
import glob
import time
import cv2 # We need OpenCV for image resizing

def normalize_map(data):
    """Normalizes a 2D numpy array to the 0.0 - 1.0 range."""
    min_val = np.min(data)
    max_val = np.max(data)
    range_val = max_val - min_val
    
    if range_val == 0:
        return np.zeros(data.shape, dtype=np.float32)
    
    return (data - min_val) / range_val

def generate_hsv_image(file_path, grid_size=64):
    """
    Generates a 64x64 HSV image from an audio file by mapping
    Loudness -> Value
    Pitch     -> Hue
    Timbre    -> Saturation
    
    Args:
        file_path (str): The full path to the audio file.
        grid_size (int): The dimension of the grid (e.g., 64).
        
    Returns:
        PIL.Image: A 3-channel RGB image, or None if error.
    """
    try:
        # 1. Load Audio
        y, sr = librosa.load(file_path, sr=None)
        
        if len(y) == 0:
            print(f"   [Warning] Skipping empty file: {os.path.basename(file_path)}")
            return None
            
        # 2. Get the full-resolution STFT
        
        # Calculate the complex STFT
        D_complex = librosa.stft(y, n_fft=2048) # <-- CHANGED
        
        # Get the magnitude spectrogram (what we need for features)
        S_full = np.abs(D_complex) # <-- CHANGED

        # --- 3. CALCULATE THE 3 FEATURE MAPS ---

        # VALUE (Loudness)
        # We use a Mel Spectrogram with 'grid_size' (64) bins
        S_mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=grid_size, n_fft=2048)
        # Convert to log scale (dB) for perceptual loudness
        V_map = librosa.amplitude_to_db(S_mel, ref=np.max)
        
        # HUE (Pitch / Spectral "Brightness")
        # We use Spectral Centroid, which finds the "center of mass" of the spectrum
        H_map_raw = librosa.feature.spectral_centroid(S=S_full, sr=sr)
        
        # SATURATION (Timbre / Tonal Purity)
        # We use Spectral Flatness. 
        # High flatness (near 1.0) = Noisy (e.g., white noise)
        # Low flatness (near 0.0) = Tonal (e.g., a sine wave)
        S_map_raw = librosa.feature.spectral_flatness(S=S_full)

        # --- 4. NORMALIZE & ALIGN MAPS ---

        # We need all maps to be (64, N_frames)
        # V_map is already (64, N_frames)
        V_map_norm = normalize_map(V_map) # Normalize 0.0-1.0

        # H_map and S_map are (1, N_frames). We repeat them 64 times
        # to match the V_map's height.
        
        # Normalize H_map (0-1)
        H_map_norm = normalize_map(H_map_raw)
        H_map_full = np.tile(H_map_norm, (grid_size, 1))
        
        # Normalize S_map (0-1) and INVERT it
        # We want tonal (low flatness) to be HIGH saturation
        S_map_norm_inv = 1.0 - normalize_map(S_map_raw)
        S_map_full = np.tile(S_map_norm_inv, (grid_size, 1))
        
        # --- 5. CREATE HSV IMAGE & RESIZE ---
        
        # Stack the 3 maps to create an HSV image
        # Shape is (64, N_frames, 3)
        hsv_image_long = np.stack([H_map_full, S_map_full, V_map_norm], axis=-1)

        # Resize the "time" axis (N_frames) to be 'grid_size' (64)
        # This squashes or stretches the audio to fit the 64-pixel width
        hsv_image_64x64 = cv2.resize(
            hsv_image_long, 
            (grid_size, grid_size), 
            interpolation=cv2.INTER_LINEAR
        )
        
        # --- 6. CONVERT TO RGB & SAVE ---
        
        # Convert from 0.0-1.0 (float) to 0-255 (uint8)
        hsv_image_uint8 = (hsv_image_64x64 * 255).astype(np.uint8)

        # Convert from HSV color space to RGB
        rgb_image_array = cv2.cvtColor(hsv_image_uint8, cv2.COLOR_HSV2RGB)
        
        # Create a PIL Image from the numpy array
        img = Image.fromarray(rgb_image_array, 'RGB')
        
        return img

    except Exception as e:
        # Catch other potential errors
        print(f"   [Error] Skipping {os.path.basename(file_path)}: {e}")
        return None


def process_all_genres(base_audio_dir, base_output_dir, grid_size=64):
    """
    (This is the same helper function from your script)
    Finds all genre sub-folders, loops through all .wav files,
    generates images, and saves them in a mirrored directory structure.
    """
    
    print(f"Starting batch processing...")
    print(f"Input directory:  {base_audio_dir}")
    print(f"Output directory: {base_output_dir}")
    print("---")
    
    start_time = time.time()
    total_files_processed = 0
    
    try:
        genre_dirs = [d.name for d in os.scandir(base_audio_dir) if d.is_dir()]
    except FileNotFoundError:
        print(f"ERROR: Input directory not found: {base_audio_dir}")
        print("Please check your 'base_audio_dir' variable.")
        return

    if not genre_dirs:
        print(f"ERROR: No genre sub-folders found in {base_audio_dir}.")
        return

    print(f"Found {len(genre_dirs)} genres: {genre_dirs}")

    # Loop over each genre folder
    for genre in genre_dirs:
        input_genre_path = os.path.join(base_audio_dir, genre)
        output_genre_path = os.path.join(base_output_dir, genre)
        
        os.makedirs(output_genre_path, exist_ok=True)
        
        audio_files = glob.glob(os.path.join(input_genre_path, "*.wav"))
        
        if not audio_files:
            print(f"\nNo .wav files found for genre: {genre}. Skipping.")
            continue

        print(f"\nProcessing {len(audio_files)} files for genre: {genre}...")
        files_in_genre = 0
        
        # Loop over each file, generate image, and save
        for file_path in audio_files:
            # Generate the new HSV image
            hsv_image = generate_hsv_image(file_path, grid_size)
            
            if hsv_image:
                base_filename = os.path.basename(file_path)
                filename_no_ext = os.path.splitext(base_filename)[0]
                output_filename = f"{filename_no_ext}.png"
                output_path = os.path.join(output_genre_path, output_filename)
                
                # Save the image
                hsv_image.save(output_path)
                files_in_genre += 1

        print(f"Finished {genre}: {files_in_genre} images saved.")
        total_files_processed += files_in_genre

    end_time = time.time()
    print("---")
    print(f"✅ All processing complete.")
    print(f"Total files processed: {total_files_processed}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    print(f"Your images are saved in the '{base_output_dir}' directory.")


# --- 🚀 HOW TO RUN ---
if __name__ == "__main__":
    
    # 1. Define your main audio directory
    # This is the folder that CONTAINS "classical", "blues", "rock", etc.
    # ****** PLEASE UPDATE THIS PATH ******
    base_audio_dir = "Data/genres_original/" 
    
    # 2. Define where you want to save the new images
    # A new folder "hsv_images_64" will be created.
    base_output_dir = "hsv_images_64/"
    
    # 3. Run the processing
    process_all_genres(base_audio_dir, base_output_dir, grid_size=64)

Starting batch processing...
Input directory:  Data/genres_original/
Output directory: hsv_images_64/
---
Found 10 genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

Processing 100 files for genre: blues...
Finished blues: 100 images saved.

Processing 100 files for genre: classical...
Finished classical: 100 images saved.

Processing 100 files for genre: country...
Finished country: 100 images saved.

Processing 100 files for genre: disco...
Finished disco: 100 images saved.

Processing 100 files for genre: hiphop...
Finished hiphop: 100 images saved.

Processing 100 files for genre: jazz...


C:\Users\geeta\AppData\Local\Temp\ipykernel_21020\82968464.py:36: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)
c:\Users\geeta\AppData\Local\Programs\Python\Python313\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


   [Error] Skipping jazz.00054.wav: 
Finished jazz: 99 images saved.

Processing 100 files for genre: metal...
Finished metal: 100 images saved.

Processing 100 files for genre: pop...
Finished pop: 100 images saved.

Processing 100 files for genre: reggae...
Finished reggae: 100 images saved.

Processing 100 files for genre: rock...
Finished rock: 100 images saved.
---
✅ All processing complete.
Total files processed: 999
Total time taken: 298.54 seconds
Your images are saved in the 'hsv_images_64/' directory.


In [4]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- 1. Configuration ---
# This MUST match the 'base_output_dir' from your generation script
IMAGE_DIR = "hsv_images_64/" 
CATEGORIES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'] # Make sure this matches your folder names
IMG_SIZE = 64 # This MUST match the 'grid_size' from your script

# --- 2. Load Data ---
print(f"Loading {IMG_SIZE}x{IMG_SIZE} images from '{IMAGE_DIR}'...")
data = []
labels = []

for i, category in enumerate(CATEGORIES):
    path = os.path.join(IMAGE_DIR, category)
    try:
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            
            # Load the image in COLOR (since your script saved as RGB)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
            
            # Resize just in case, though your script should handle this
            img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append(img_resized)
            labels.append(i) # 0 for blues, 1 for rock, 2 for nature
    except Exception as e:
        print(f"Error loading images from {path}: {e}")
        print("Please double-check your 'IMAGE_DIR' and 'CATEGORIES'")

if not data:
    print("\nError: No data loaded. Stopping.")
else:
    # --- 3. Preprocessing ---
    # Convert to numpy arrays
    X = np.array(data)
    y = np.array(labels)

    # Normalize pixel values from 0-255 to 0.0-1.0
    # This is critical for neural network performance
    X = X / 255.0

    # --- 4. Split Data (80% train, 20% test) ---
    # stratify=y ensures the 80/20 split has a fair mix of all 3 classes
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"\nTotal samples loaded: {len(X)}")
    print(f"Training data shape: {X_train.shape}") # Should be (240, 64, 64, 3)
    print(f"Testing data shape: {X_test.shape}")   # Should be (60, 64, 64, 3)

    # --- 5. Build the CNN Model ---
    # For 64x64 images, we'll use two Conv/Pool blocks
    
    model = tf.keras.models.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))) 
    
    # Block 1:
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    # Block 2:
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    # Flatten: Squashes 2D data to 1D
    model.add(tf.keras.layers.Flatten())
    
    # Hidden layer:
    model.add(tf.keras.layers.Dense(128, activation='relu')) # Increased size for 64x64
    
    # Output layer: 3 neurons (1 per class), softmax for probabilities
    model.add(tf.keras.layers.Dense(len(CATEGORIES), activation='softmax'))

    model.summary()

    # --- 6. Compile the Model ---
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy', # Use this for integer labels (0, 1, 2)
        metrics=['accuracy']
    )

    # --- 7. Train the Model ---
    print("\nTraining model...")
    history = model.fit(
        X_train, y_train,
        epochs=30, # 30-50 epochs is a good starting point
        validation_data=(X_test, y_test),
        verbose=1
    )

    # --- 8. Evaluate ---
    print("\nEvaluating on test data...")
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print("---" * 10)
    print(f"✅ Final Test Accuracy: {test_acc * 100:.2f}%")
    print("---" * 10)

Loading 64x64 images from 'hsv_images_64/'...

Total samples loaded: 999
Training data shape: (799, 64, 64, 3)
Testing data shape: (200, 64, 64, 3)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,442 (6.20 MB)

 Trainable params: 1,626,442 (6.20 MB)

 Non-trainable params: 0 (0.00 B)


Training model...
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 184ms/step - accuracy: 0.1727 - loss: 2.1796 - val_accuracy: 0.2750 - val_loss: 2.0587
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 141ms/step - accuracy: 0.3091 - loss: 1.8501 - val_accuracy: 0.3750 - val_loss: 1.8510
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.5144 - loss: 1.4897 - val_accuracy: 0.3750 - val_loss: 1.8582
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.6095 - loss: 1.1898 - val_accuracy: 0.4200 - val_loss: 1.7571
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.6496 - loss: 1.0285 - val_accuracy: 0.4650 - val_loss: 1.5250
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - accuracy: 0.7835 - loss: 0.7361 - val_accuracy: 0.4800 - val_loss: 1.6634
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 141ms/step - accuracy: 0.8373 - loss: 0.5579 - val_accuracy: 0.4950 - val_loss: 1.6301
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 151ms/step - accuracy: 0.9049 - loss: 0.401

In [5]:
import librosa
import numpy as np
import cv2
import soundfile as sf  # We use soundfile to save the audio
from PIL import Image

def generate_audio_from_image(image_path, n_fft=2048, grid_size=64, sr=22050):
    """
    Reconstructs an audio waveform from a 64x64 image.
    
    This baseline method uses ONLY the V (Value) channel 
    as a Mel Spectrogram and estimates phase.
    """
    
    try:
        # --- 1. Load Image ---
        # Load the 64x64 RGB image
        img_rgb = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        # --- 2. Convert RGB -> HSV ---
        # Convert to HSV so we can access the Value channel
        hsv_image = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        
        # --- 3. Extract the V (Value) Channel ---
        # hsv_image[:, :, 2] is the V channel
        # Values are 0-255. We normalize to 0.0-1.0 float.
        V_map_norm = hsv_image[:, :, 2] / 255.0
        
        # --- 4. Un-normalize the V Channel ---
        # In our forward script, V was a log-Mel spectrogram from librosa.amplitude_to_db,
        # which maps to a range of approx. -80dB (silence) to 0dB (max).
        # We now reverse that mapping.
        # 0.0 -> -80dB
        # 1.0 -> 0dB
        V_map_db = (V_map_norm * 80.0) - 80.0
        
        # --- 5. Convert dB -> Amplitude ---
        # This is our approximate 64x64 Mel spectrogram in amplitude
        S_mel_approx = librosa.db_to_amplitude(V_map_db)
        
        # --- 6. Invert Mel Spectrogram -> Linear STFT ---
        # This is the "magic" step. We ask librosa to "best-guess"
        # what a full STFT (1025 x 64) would result in this
        # 64x64 Mel spectrogram.
        S_stft_approx = librosa.feature.inverse.mel_to_stft(
            S_mel_approx, 
            sr=sr, 
            n_fft=n_fft
        )
        
        # --- 7. Reconstruct Audio with Griffin-Lim ---
        # This algorithm iteratively "guesses" the phase
        # that is most consistent with the magnitude spectrogram.
        print("   Running Griffin-Lim... (this can take a moment)")
        y_reconstructed = librosa.griffinlim(S_stft_approx)
        
        return y_reconstructed

    except Exception as e:
        print(f"   [Error] Could not process image {image_path}: {e}")
        return None

# --- 🚀 HOW TO RUN ---
if __name__ == "__main__":
    
    # 1. Choose an image you just created
    # ****** PLEASE UPDATE THIS PATH ******
    image_to_test = "hsv_images_64/rock/rock.00000.png"
    
    # 2. Define where to save the new audio file
    # ****** PLEASE UPDATE THIS PATH ******
    output_audio_path = "reconstructed_audio_test.wav"
    
    # 3. Define the sample rate (must match your original audio)
    SR = 22050 
    
    print(f"Attempting to reconstruct audio from: {image_to_test}")
    
    # Generate the audio
    reconstructed_waveform = generate_audio_from_image(image_to_test, sr=SR)
    
    if reconstructed_waveform is not None:
        # Save the audio
        sf.write(output_audio_path, reconstructed_waveform, SR)
        print(f"\n✅ Success! Reconstructed audio saved to:")
        print(f"   {output_audio_path}")
        print("\nListen to it! Compare it to the original 'rock.00000.wav'.")
    else:
        print("\n❌ Failed to generate audio.")

Attempting to reconstruct audio from: hsv_images_64/rock/rock.00000.png
   Running Griffin-Lim... (this can take a moment)

✅ Success! Reconstructed audio saved to:
   reconstructed_audio_test.wav

Listen to it! Compare it to the original 'rock.00000.wav'.


In [ ]:
import librosa
import numpy as np
import cv2
import soundfile as sf
from PIL import Image

def generate_audio_from_image(image_path,
                              audio_duration_sec=30,
                              sr=22050,
                              n_fft=2048,
                              hop_length=512, # We still need this for Griffin-Lim
                              grid_size=64):
    """
    Reconstructs a full-duration audio waveform from a 64x64 image.
    Stretches the time axis from 64 pixels back to its original frame count.
    """
    
    try:
        # --- 1. Load Image ---
        img_rgb = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        # --- 2. Convert RGB -> HSV ---
        hsv_image = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        
        # --- 3. Extract the V (Value) Channel ---
        V_map_norm = hsv_image[:, :, 2] / 255.0
        
        # --- 4. Un-normalize the V Channel ---
        V_map_db = (V_map_norm * 80.0) - 80.0
        
        # --- 5. STRETCH THE TIME AXIS ---
        n_frames_original = int(np.ceil(audio_duration_sec * sr / hop_length))
        
        print(f"   Stretching {grid_size} frames back to {n_frames_original} frames...")
        
        V_map_db_stretched = cv2.resize(
            V_map_db,
            (n_frames_original, grid_size), # (width, height)
            interpolation=cv2.INTER_LINEAR
        )
        
        # --- 6. Convert dB -> Amplitude ---
        S_mel_approx = librosa.db_to_amplitude(V_map_db_stretched)
        
        # --- 7. Invert Mel Spectrogram -> Linear STFT ---
        # The hop_length argument is removed from this call
        S_stft_approx = librosa.feature.inverse.mel_to_stft(
            S_mel_approx,
            sr=sr,
            n_fft=n_fft
            # hop_length=hop_length <-- THIS WAS THE BUG
        )
        
        # --- 8. Reconstruct Audio with Griffin-Lim ---
        print("   Running Griffin-Lim... (this will take longer now)")
        # We *do* need hop_length here
        y_reconstructed = librosa.griffinlim(
            S_stft_approx,
            hop_length=hop_length 
        )
        
        return y_reconstructed

    except Exception as e:
        print(f"   [Error] Could not process image {image_path}: {e}")
        return None

# --- 🚀 HOW TO RUN ---
if __name__ == "__main__":
    
    image_to_test = "hsv_images_64/rock/rock.00013.png"
    output_audio_path = "reconstructed_audio_test_30sec_rock_13.wav"
    SR = 22050
    HOP_LEN = 512 # Define hop_length explicitly
    
    print(f"Attempting to reconstruct audio from: {image_to_test}")
    
    # Generate the audio
    reconstructed_waveform = generate_audio_from_image(
        image_to_test, 
        audio_duration_sec=30,
        sr=SR,
        hop_length=HOP_LEN
    )
    
    if reconstructed_waveform is not None:

        print("   Normalizing audio amplitude...")
        max_val = np.max(np.abs(reconstructed_waveform))
        if max_val > 0:
             reconstructed_waveform = reconstructed_waveform / max_val
        
        # Save the audio
        sf.write(output_audio_path, reconstructed_waveform, SR)
        print(f"\n✅ Success! Reconstructed 30s audio saved to:")
        print(f"   {output_audio_path}")
        print("\nListen to it! It will sound 'phasey', but the tempo should be correct.")
    else:
        print("\n❌ Failed to generate audio.")

Attempting to reconstruct audio from: hsv_images_64/rock/rock.00013.png
   Stretching 64 frames back to 1292 frames...
   Running Griffin-Lim... (this will take longer now)

✅ Success! Reconstructed 30s audio saved to:
   reconstructed_audio_test_30sec_rock_13.wav

Listen to it! It will sound 'phasey', but the tempo should be correct.


In [6]:
import os
import cv2
import numpy as np

def analyze_image_directory(directory_path):
    """
    Scans a directory to count all images, get the dimensions
    of the first one, and calculate its total pixel count.
    
    Args:
        directory_path (str): The path to the main folder 
                              (e.g., "hsv_images_64/").
    """
    
    total_images = 0
    image_dimensions = None
    total_pixels = 0
    
    print(f"Analyzing directory: {directory_path}\n")
    
    if not os.path.isdir(directory_path):
        print(f"Error: Directory not found at '{directory_path}'")
        return

    # os.walk will go through the main folder and all its sub-folders
    for root, dirs, files in os.walk(directory_path):
        for file_name in files:
            # Check for common image extensions
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                
                # --- Count the image ---
                total_images += 1
                
                # --- Get dimensions and pixel count of the first image ---
                if image_dimensions is None:
                    try:
                        full_path = os.path.join(root, file_name)
                        image = cv2.imread(full_path)
                        
                        if image is not None:
                            # .shape returns (height, width, channels)
                            h, w, c = image.shape
                            image_dimensions = (h, w, c)
                            
                            # --- Calculate total pixels ---
                            total_pixels = h * w * c
                        else:
                            print(f"Warning: Could not read first image: {full_path}")
                            
                    except Exception as e:
                        print(f"Error reading dimensions from {full_path}: {e}")

    # --- Print the final report ---
    if total_images == 0:
        print("No images found in that directory.")
    else:
        print("--- Analysis Complete ---")
        print(f"✅ Total Number of Images Found: {total_images}")
        
        if image_dimensions:
            height, width, channels = image_dimensions
            print(f"✅ Pixel-wise Dimensions (H x W x C): {height} x {width} x {channels}")
            print(f"✅ Total Number of Pixels (H * W * C): {total_pixels} (data points)")
        else:
            print("❌ Could not determine image dimensions.")

# --- 🚀 HOW TO RUN ---
if __name__ == "__main__":
    
    # *** UPDATE THIS PATH ***
    # This should be the main folder that CONTAINS 
    # your 'blues', 'rock', etc. sub-folders.
    IMAGE_DIR = "hsv_images_64/" 
    
    analyze_image_directory("Data/images_original/rock/")
    analyze_image_directory("hsv_images_64/rock/")


Analyzing directory: Data/images_original/rock/

--- Analysis Complete ---
✅ Total Number of Images Found: 100
✅ Pixel-wise Dimensions (H x W x C): 288 x 432 x 3
✅ Total Number of Pixels (H * W * C): 373248 (data points)
Analyzing directory: hsv_images_64/rock/

--- Analysis Complete ---
✅ Total Number of Images Found: 100
✅ Pixel-wise Dimensions (H x W x C): 64 x 64 x 3
✅ Total Number of Pixels (H * W * C): 12288 (data points)
